In [5]:
library(jsonlite)
library(tidyr)
library(ggplot2)
library(testthat)

In [6]:
currency_code_vector<-c('AED', 'AFN', 'ALL', 'AMD', 'ANG', 'AOA', 'ARS', 'AUD', 'AWG', 'AZN',
                        'BAM', 'BBD', 'BDT', 'BGN', 'BHD', 'BIF', 'BMD', 'BND', 'BOB', 'BRL', 'BSD', 'BTC', 'BTN', 'BWP', 'BYN', 'BZD',
                        'CAD', 'CDF', 'CHF', 'CLF', 'CLP', 'CNH', 'CNY', 'COP','CRC', 'CUC', 'CUP','CVE','CZK',
                        'DJF', 'DKK', 'DOP', 'DZD',
                        'EGP', 'ERN', 'ETB', 'EUR',
                        'FJD', 'FKP',
                        'GBP', 'GEL', 'GGP', 'GHS', 'GIP', 'GMD', 'GNF', 'GTQ', 'GYD',
                        'HKD', 'HNL', 'HRK', 'HTG', 'HUF', 
                        'IDR', 'ILS', 'IMP', 'INR', 'IQD', 'IRR', 'ISK',
                        'JEP', 'JMD', 'JOD', 'JPY',
                        'KES', 'KGS', 'KHR', 'KMF', 'KPW', 'KRW', 'KWD', 'KYD', 'KZT',
                        'LAK', 'LBP', 'LKR', 'LRD', 'LSL', 'LYD',
                        'MAD', 'MDL', 'MGA', 'MKD', 'MMK', 'MNT', 'MOP', 'MRU', 'MUR', 'MVR', 'MWK', 'MXN', 'MYR', 'MZN',
                        'NAD', 'NGN', 'NIO', 'NOK', 'NPR', 'NZD', 
                        'OMR',
                        'PAB', 'PEN', 'PGK', 'PHP', 'PKR', 'PLN', 'PYG',
                        'QAR',
                        'RON', 'RSD', 'RUB', 'RWF',
                        'SAR', 'SBD', 'SCR', 'SDG', 'SEK', 'SGD', 'SHP', 'SLL', 'SOS', 'SRD', 'SSP', 'STD', 'STN', 'SVC', 'SYP', 'SZL',
                        'THB', 'TJS', 'TMT', 'TND', 'TOP', 'TRY', 'TTD', 'TWD', 'TZS',
                        'UAH', 'UGX', 'USD', 'UYU', 'UZS', 
                        'VES', 'VND', 'VUV', 
                        'WST', 
                        'XAF', 'XAG', 'XAU', 'XCD', 'XDR', 'XOF', 'XPD', 'XPF', 'XPT',
                        'YER',
                        'ZAR', 'ZMW', 'ZWL')


    A wrapper function to retrieve information about how a list of currencies fluctuate on a specific base at a specific amount on a day-to-day basis from the Exchange rates API. The maximum allowed timeframe is 366 days.
    
    Parameters:
    -----------
    start_date: The start date of your preferred fluctuation timeframe (e.g. start='2020-01-01'). 
    end_date: The end date of your preferred fluctuation timeframe (e.g. end='2020-01-01').
    base: Changing base currency. Enter the three-letter currency code of your preferred base currency (e.g. base='USD').
    symbols: Enter a list of comma-separated currency codes to limit output currencies (e.g., symbols=c('USD','EUR','CZK')]
    amount: The amount to be converted (e.g., amount=1200).

    Returns:
    --------
    A dataframe with the columns `Start`, `End`, `Base`, `Rates.start_rate`, `Rates.end_rate`, `Rates.change_pct`, and `Amount`.
    Start: Start date.
    End: End date.
    Base: Changing base currency.
    Rates.start_rate: Rates at the start date.
    Rates.end_rate: Rates at the end date.
    Rates.change: Rates.start_rate-Rates.end_rate.
    Rates.change_pct: (Rates.start_rate-Rates.end_rate)/Rates.start_rate
    Amount: The amount to be converted
    
    


In [7]:
fluctuation <- function(start, end, base='EUR', symbols='', amount=1) {
    if(base %in% currency_code_vector == FALSE){
        warning("Invalid base currency name in base!")
        return ("Error")
        }

    if(all(unlist(strsplit(symbols, ",")) %in% currency_code_vector) == FALSE){
        warning("Invalid currency names in symbols!")
        return ("Error")
        }

    if(is.numeric(amount) == FALSE){
        warning("Input a number!")
        return ("Error")
        }

    url<-paste('https://api.exchangerate.host/fluctuation?start_date=',start,'&end_date=',end,'&base=',base,'&symbols=',symbols,'&amount=',amount,"&places=2",sep="")
    data <- fromJSON(url)

    if(data$success!=TRUE){
        warning("Error in connecting to the API!")
        return ("Error")
    }

    if(length(data$rates)==0){
        warning("Error in the parameter, please check!")
        return ("Error")
    }
    
    start <- as.Date(start)
    end <- as.Date(end)
    if(difftime(end, start, units = "days")>366){
        warning("Error in the timeframe! The maximum allowed is 366 days!")
        return ("Error")
    }

    data <- data.frame('Start' = data$start_date, 'End' = data$end_date, 'Base' = base, 'Amount' = amount, 'Rates' = do.call(rbind, data$rates))
    return (data)
}

In [8]:
#source("../R/fluctuation.R")

test_that("fluctuation works", {
    
    expect_equal(typeof(fluctuation('2022-01-01','2022-02-10', symbols='USD')),'list')
    expect_equal(ncol(fluctuation('2022-01-01','2022-02-10', symbols='USD')),1,4) 
    
    url<-'https://api.exchangerate.host/fluctuation?start_date=2022-01-01&end_date=2022-02-10&base=EUR&symbols=USD&amount=1&places=2'
    data <- fromJSON(url)
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[1]],data$start_date[[1]])
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[2]],data$end_date[[1]])
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[3]],'EUR')    
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[4]],1)
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[5]][[1]],data$rates[[1]][[1]])
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[6]][[1]],data$rates[[1]][[2]])
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[7]][[1]],data$rates[[1]][[3]])
    expect_equal(fluctuation('2022-01-01','2022-02-10', symbols='USD')[[8]][[1]],data$rates[[1]][[4]])
        
    expect_equal(typeof(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)),'list')
    expect_equal(ncol(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)), 2,8) 
    
    url<-'https://api.exchangerate.host/fluctuation?start_date=2021-02-01&end_date=2021-05-10&base=CNH&symbols=USD,CAD&amount=1000&places=2'
    data <- fromJSON(url)
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[1]][1],data$start_date)
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[2]][1],data$end_date)
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[3]][1],'CNH')
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[4]][1],1000)
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[5]][[1]],data$rates[[1]][[1]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[5]][[2]],data$rates[[2]][[1]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[6]][[1]],data$rates[[1]][[2]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[6]][[2]],data$rates[[2]][[2]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[7]][[1]],data$rates[[1]][[3]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[7]][[2]],data$rates[[2]][[3]])
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[8]][[1]],data$rates[[1]][[4]])     
    expect_equal(fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD', amount=1000)[[8]][[2]],data$rates[[2]][[4]])     

})

test_that("fluctuation throw errors and get warnings", {
    expect_warning(fluctuation('2021-02-01','2021-05-10','MMM'))
    expect_warning(fluctuation('2021-02-01','2021-05-10',symbols='MMM'))
    expect_warning(fluctuation('2021-02-01','2021-05-10',amount='MMM'))
    expect_warning(fluctuation('2021-02-01','2022-05-10'))    
})

Test passed 🎊
Test passed 😀


In [9]:
fluctuation('2021-02-01','2021-05-10', base='CNH', symbols='USD,CAD,JPY,HKD,GBP', amount=1000)

,Start,End,Base,Amount,Rates.start_rate,Rates.end_rate,Rates.change,Rates.change_pct
,<chr>,<chr>,<chr>,<dbl>,<named list>,<named list>,<named list>,<named list>
CAD,2021-02-01,2021-05-10,CNH,1000,198.52,188.26,10.26,0.05
GBP,2021-02-01,2021-05-10,CNH,1000,113.05,110.21,2.84,0.03
HKD,2021-02-01,2021-05-10,CNH,1000,1197.74,1209.15,-11.41,-0.01
JPY,2021-02-01,2021-05-10,CNH,1000,16213.62,16942.69,-729.07,-0.04
USD,2021-02-01,2021-05-10,CNH,1000,154.49,155.7,-1.21,-0.01
